# Import libraries

In [1]:
# !pip install ccxt 
# !pip install pandas
# !pip install matplotlib


Defaulting to user installation because normal site-packages is not writeable
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
   ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
   ---------------- ----------------------- 2.4/5.7 MB 12.2 MB/s eta 0:00:01
   -------------------------------------- - 5.5/5.7 MB 13.4 MB/s eta 0:00:01
   ---------------------------------------- 5.7/5.7 MB 12.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 3.2/3.2 MB 27.0 MB/s eta 0:00:00
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 20.6 MB/s eta 0:00:00
Using cached idna-3.10-py3-none-any.whl (70 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Load API keys 

# Load exchange 

# Function for executing trades

Testing

In [ ]:
import ccxt
import time
import pandas as pd
from datetime import datetime

# Binance API keys (Replace with your actual API keys)
BINANCE_API_KEY = 'your_api_key'
BINANCE_SECRET_KEY = 'your_secret_key'

# Initialize Binance Futures API
binance = ccxt.binance({
    'apiKey': BINANCE_API_KEY,
    'secret': BINANCE_SECRET_KEY,
    'options': {'defaultType': 'future'}
})

# Settings
SYMBOL = 'BTC/USDT'
LEVERAGE = 5
TRADE_AMOUNT = 0.01  # Adjust based on risk
FUNDING_THRESHOLD = 0.0005  # Arbitrage trigger threshold
DATA_FILE = "funding_arbitrage_data.csv"

# Store trade history
columns = ["timestamp", "funding_rate", "position", "profit", "balance"]
df = pd.DataFrame(columns=columns)


# Function to fetch funding rate
def get_funding_rate(symbol):
    try:
        funding_rate = binance.fetch_funding_rate(symbol)
        return funding_rate['fundingRate']
    except Exception as e:
        print(f"Error fetching funding rate: {e}")
        return None


# Function to place trades
def place_order(symbol, side, amount, leverage=LEVERAGE):
    try:
        binance.set_leverage(leverage, symbol)
        order = binance.create_market_order(symbol, side, amount)
        print(f"Order placed: {side} {amount} {symbol}")
        return order
    except Exception as e:
        print(f"Error placing order: {e}")
        return None


# Function to close positions
def close_positions(symbol, amount):
    place_order(symbol, "sell", amount)  # Close long position
    place_order(symbol, "buy", amount)  # Close short position


# Main strategy function
def funding_arbitrage():
    global df

    funding_rate = get_funding_rate(SYMBOL)
    if funding_rate is None:
        return

    print(f"Current funding rate: {funding_rate}")

    if funding_rate > FUNDING_THRESHOLD:
        print("Arbitrage opportunity detected: LONG position opened.")
        order = place_order(SYMBOL, "buy", TRADE_AMOUNT)
        position = "long"
    elif funding_rate < -FUNDING_THRESHOLD:
        print("Arbitrage opportunity detected: SHORT position opened.")
        order = place_order(SYMBOL, "sell", TRADE_AMOUNT)
        position = "short"
    else:
        print("No arbitrage opportunity found.")
        return

    # Wait for funding period (8 hours)
    time.sleep(8 * 60 * 60)

    # Close the trade
    close_positions(SYMBOL, TRADE_AMOUNT)

    # Log trade data
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    profit = TRADE_AMOUNT * funding_rate * 8  # Approximate funding profit
    balance = binance.fetch_balance()['total']['USDT']

    new_entry = pd.DataFrame([[timestamp, funding_rate, position, profit, balance]], columns=columns)
    df = pd.concat([df, new_entry], ignore_index=True)

    # Save to CSV for later comparison
    df.to_csv(DATA_FILE, index=False)
    print("Trade logged successfully.\n")


# Run the script periodically (Every 8 hours)
while True:
    funding_arbitrage()


## Funding rate trading bot

In [ ]:
import ccxt
import time
import pandas as pd
from datetime import datetime

# Binance Testnet API keys (Replace with your actual Testnet API keys)
BINANCE_API_KEY = 'your_testnet_api_key'
BINANCE_SECRET_KEY = 'your_testnet_secret_key'

# Connect to Binance Testnet
binance = ccxt.binance({
    'apiKey': BINANCE_API_KEY,
    'secret': BINANCE_SECRET_KEY,
    'options': {'defaultType': 'future'},  # Use futures trading
    'urls': {'api': 'https://testnet.binancefuture.com'}  # Testnet URL
})

# Settings
SYMBOL = 'BTC/USDT'
LEVERAGE = 5
TRADE_AMOUNT = 0.01  # Adjust based on risk
FUNDING_THRESHOLD = 0.0005  # Arbitrage trigger threshold
DATA_FILE = "funding_arbitrage_data_testnet.csv"

# Store trade history
columns = ["timestamp", "funding_rate", "position", "profit", "balance"]
df = pd.DataFrame(columns=columns)


# Function to fetch funding rate
def get_funding_rate(symbol):
    try:
        funding_rate = binance.fetch_funding_rate(symbol)
        return funding_rate['fundingRate']
    except Exception as e:
        print(f"Error fetching funding rate: {e}")
        return None


# Function to place trades
def place_order(symbol, side, amount, leverage=LEVERAGE):
    try:
        binance.set_leverage(leverage, symbol)
        order = binance.create_market_order(symbol, side, amount)
        print(f"Testnet Order placed: {side} {amount} {symbol}")
        return order
    except Exception as e:
        print(f"Error placing testnet order: {e}")
        return None


# Function to close positions
def close_positions(symbol, amount):
    place_order(symbol, "sell", amount)  # Close long position
    place_order(symbol, "buy", amount)  # Close short position


# Main strategy function
def funding_arbitrage():
    global df

    funding_rate = get_funding_rate(SYMBOL)
    if funding_rate is None:
        return

    print(f"Current Testnet Funding Rate: {funding_rate}")

    if funding_rate > FUNDING_THRESHOLD:
        print("Arbitrage opportunity detected: LONG position opened.")
        order = place_order(SYMBOL, "buy", TRADE_AMOUNT)
        position = "long"
    elif funding_rate < -FUNDING_THRESHOLD:
        print("Arbitrage opportunity detected: SHORT position opened.")
        order = place_order(SYMBOL, "sell", TRADE_AMOUNT)
        position = "short"
    else:
        print("No arbitrage opportunity found.")
        return

    # Wait for funding period (8 hours)
    time.sleep(8 * 60 * 60)

    # Close the trade
    close_positions(SYMBOL, TRADE_AMOUNT)

    # Log trade data
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    profit = TRADE_AMOUNT * funding_rate * 8  # Approximate funding profit
    balance = binance.fetch_balance()['total']['USDT']

    new_entry = pd.DataFrame([[timestamp, funding_rate, position, profit, balance]], columns=columns)
    df = pd.concat([df, new_entry], ignore_index=True)

    # Save to CSV for later comparison
    df.to_csv(DATA_FILE, index=False)
    print("Testnet Trade logged successfully.\n")


# Run the script periodically (Every 8 hours)
while True:
    funding_arbitrage()


# Back testing portion


In [2]:
!pip install pandas
!pip install matplotlib

  Using cached pandas-2.2.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.2-cp313-cp313-win_amd64.whl.metadata (60 kB)
Using cached pandas-2.2.3-cp313-cp313-win_amd64.whl (11.5 MB)
Using cached numpy-2.2.2-cp313-cp313-win_amd64.whl (12.6 MB)
  Using cached contourpy-1.3.1-cp313-cp313-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ------------- -------------------------- 2.6/8.0 MB 12.6 MB/s eta 0:00:01
   ----------------------------------- ---- 7.1/8.0 MB 17.4 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 17.1 MB/s eta 0:00:00
Using cached contourpy-1.3.1-cp313-cp313-win_amd64.whl (220 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 26.3 MB/s eta 0:00:00
   -------------------------------

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load historical funding rate data (CSV from Binance API or past logs)
# Ensure the CSV has a "timestamp" and "funding_rate" column
DATA_FILE = "historical_funding_rates.csv" # change 
df = pd.read_csv(DATA_FILE)

# Settings for backtesting
TRADE_AMOUNT = 0.01  # Amount per trade
FUNDING_THRESHOLD = 0.0005  # Arbitrage trigger threshold
LEVERAGE = 5

# Add columns for backtesting results
df["position"] = "none"
df["profit"] = 0

# Simulate trading based on past funding rates
for i in range(len(df) - 1):
    funding_rate = df.loc[i, "funding_rate"]

    if funding_rate > FUNDING_THRESHOLD:
        df.loc[i, "position"] = "long"
        df.loc[i, "profit"] = TRADE_AMOUNT * funding_rate * 8  # Funding profit estimate
    elif funding_rate < -FUNDING_THRESHOLD:
        df.loc[i, "position"] = "short"
        df.loc[i, "profit"] = TRADE_AMOUNT * abs(funding_rate) * 8  # Funding profit estimate
    else:
        df.loc[i, "position"] = "none"

# Calculate total profit
total_profit = df["profit"].sum()
print(f"Total Estimated Profit from Backtest: ${total_profit:.2f}")

# Plot Funding Rate vs. Profit
plt.figure(figsize=(12, 5))
plt.plot(df["timestamp"], df["funding_rate"], label="Funding Rate", color="blue")
plt.axhline(y=FUNDING_THRESHOLD, color="green", linestyle="--", label="Long Threshold")
plt.axhline(y=-FUNDING_THRESHOLD, color="red", linestyle="--", label="Short Threshold")
plt.legend()
plt.title("Funding Rate Over Time")
plt.xlabel("Timestamp")
plt.ylabel("Funding Rate")
plt.xticks(rotation=45)
plt.show()

# Save results for later analysis
df.to_csv("funding_arbitrage_backtest_results.csv", index=False)
